In [103]:
import pandas as pd

In [104]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import pandas as pd
import panel as pn
import panel.widgets as pnw

pn.extension("plotly", "tabulator", theme='dark')
pn.extension()

In [105]:
# transactions data
transactions = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/transactions_cleaned.csv')
transactions['Transaction Date'] = pd.to_datetime(df['Transaction Date'])
transactions['Publication Date'] = pd.to_datetime(df['Publication Date'])

In [106]:
# trades data
trades = pd.read_csv('https://raw.githubusercontent.com/maxdokukin/Politician-Trades/main/Data/data/trades.csv')

In [107]:
# global vars
party_colors = {
        'Democrat': '#0015BC',
        'Republican': '#DE0100',
        'Other': '#777777',
    }

name_info_dictionary = transactions[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates()

In [108]:
# transactions tab
group_by_selector = pnw.Select(options=['Politician Name', 'Party', 'Chamber', 'State', 'Issuer Name'])

plot_type_selector = pn.widgets.RadioButtonGroup(options=["Bar Plot", "Pie Chart"])
plot_functions = {"Bar Plot": px.bar, "Pie Chart": px.pie}

show_entries_count = pnw.EditableIntSlider(start=10, end=50, step=1, value=10)


def update_transactions_plot(event):
    col = group_by_selector.value
    plot_func = plot_functions[plot_type_selector.value]
    data = transactions.groupby(col).size().reset_index(name='Transaction Count').sort_values('Transaction Count').tail(show_entries_count.value)

    if plot_type_selector.value == 'Pie Chart':
        if col == 'Politician Name':
            data = data.merge(name_info_dictionary, on='Politician Name')
            fig = plot_func(data, names='Politician Name', values='Transaction Count', color='Party', color_discrete_map=party_colors)

        elif col == 'Party':
            fig = plot_func(data, names='Party', values='Transaction Count', color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, names=col, values='Transaction Count')

    else:
        if col == 'Politician Name':
            data = data.merge(name_info_dictionary, on='Politician Name')
            fig = plot_func(data, x='Transaction Count', y='Politician Name', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        elif col == 'Party':
            fig = plot_func(data, x='Transaction Count', y='Party', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        else:
            fig = plot_func(data, x=data['Transaction Count'], y=data.columns[0])

        fig.update_yaxes(categoryorder='total ascending')

    fig.update_layout(
        title={
            'text': "Distribution of Transaction Count by " + col,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )

    transactions_plot_pane.object = fig


group_by_selector.param.watch(update_transactions_plot, 'value')
plot_type_selector.param.watch(update_transactions_plot, 'value')
show_entries_count.param.watch(update_transactions_plot, 'value')

transactions_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)

update_transactions_plot(None)

transaction_count_analysis = pn.Column(pn.Row("# Transaction Count Analysis"),
                                      pn.Row("Group by", group_by_selector),
                                      pn.Row("Show Top", show_entries_count),
                                      pn.Row("Plot Type", plot_type_selector),
                                      transactions_plot_pane
                                      ,name= 'Transaction Count Analysis'
                                      )
# TODO: Fix Pie Chart Hover Data

In [109]:
# trades tab
trades_tab_selector = pn.widgets.RadioButtonGroup(options=["Trades Profitability Distribution", "Average Profit by Politician", "Profitability Scatterplot"])


# x axis scale slider
range_slider = pnw.RangeSlider()
min_value_input = pnw.IntInput(name='Minimum Value (x)')
max_value_input = pnw.IntInput(name='Maximum Value (x)')
update_range_button = pnw.Button(name='Update Range (x-axis)')

def update_slider_from_inputs(event):
    range_slider.value = (int(min_value_input.value), int(max_value_input.value))

def update_inputs_from_slider(event):
    min_value_input.value, max_value_input.value = int(range_slider.value[0]), int(range_slider.value[1])

def update_slider_range(min_val, max_val, step):

    range_slider.start = min_val
    range_slider.end = max_val
    range_slider.step = step
    range_slider.value = (min_val, max_val)

    min_value_input.start = min_val
    min_value_input.end = max_val
    min_value_input.step = step
    min_value_input.value = min_val

    max_value_input.start = min_val
    max_value_input.end = max_val
    max_value_input.step = step
    max_value_input.value = max_val


# plot updates
def update_trades_plot(event):
    print(event)
    if trades_tab_selector.value == 'Trades Profitability Distribution':
        if event is not None and event.name != 'clicks': #update range button was pressed. do not reset the range
            update_slider_range(-2900, 3200, 1)

        data = trades[(trades['Annualized Percentage Profit'] >= min_value_input.value) & (trades['Annualized Percentage Profit'] <= max_value_input.value)]
        fig = px.histogram(data, x='Annualized Percentage Profit', color_discrete_sequence=['#1B4242'])

    elif trades_tab_selector.value == 'Average Profit by Politician':
        if event is not None and event.name != 'clicks':
            update_slider_range(1, 78, 1)

        data = trades.groupby('Politician Name')['Annualized Percentage Profit'].mean().sort_values(ascending=False).reset_index(name='Average Annualized Percentage Profit')
        data = data.merge(transactions[['Politician Name', 'Party', 'Chamber', 'State']].drop_duplicates(), on='Politician Name')
        data = data.merge(trades.groupby('Politician Name').size().reset_index(name='Total Trade Cycles').drop_duplicates(), on='Politician Name')
        data = data.iloc[min_value_input.value - 1:max_value_input.value]

        fig = px.bar(data, x='Average Annualized Percentage Profit', y='Politician Name', hover_data=data.columns, color='Party', color_discrete_map=party_colors)
        fig.update_yaxes(categoryorder='total ascending')
    else:
        if event is not None and event.name != 'clicks': #update range button was pressed. do not reset the range
            update_slider_range(-260, 700, 1)

        data = trades.groupby('Politician Name')['Annualized Percentage Profit'].mean().sort_values(ascending=False).reset_index(name='Average Annualized Percentage Profit')
        data = data.merge(name_info_dictionary, on='Politician Name')
        data = data.merge(trades.groupby('Politician Name').size().reset_index(name='Total Trade Cycles').drop_duplicates(), on='Politician Name')
        data = data[(data['Average Annualized Percentage Profit'] >= min_value_input.value) & (data['Average Annualized Percentage Profit'] <= max_value_input.value)]

        fig = px.scatter(data, x='Average Annualized Percentage Profit', y='Total Trade Cycles', hover_data=data.columns, color='Party', color_discrete_map=party_colors)

    fig.update_layout(
        title={
            'text': trades_tab_selector.value,
            'y':0.97,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        }
    )
    trades_plot_pane.object = fig


# dynamic updates
trades_tab_selector.param.watch(update_trades_plot, 'value')

min_value_input.param.watch(update_slider_from_inputs, 'value')
max_value_input.param.watch(update_slider_from_inputs, 'value')
range_slider.param.watch(update_inputs_from_slider, 'value')
update_range_button.on_click(update_trades_plot)


# initiate
trades_plot_pane = pn.pane.Plotly(sizing_mode='stretch_width', height=600)
update_slider_range(-2900, 3200, 1)
update_trades_plot(None)


# group
trade_cycle_analysis = pn.Column(pn.Row("# Trade Cycle Analysis"),
                                    #   pn.Row("Group by", group_by_selector),
                                      pn.Row("Plot Type", trades_tab_selector),
                                      pn.Row(min_value_input, range_slider, max_value_input, pn.Column(pn.Spacer(height=17), update_range_button)),
                                      trades_plot_pane
                                      ,name= 'Trade Cycle Analysis'
                                      )

None


In [110]:
tabs = pn.Tabs(transaction_count_analysis,
               trade_cycle_analysis
            #    performance_analysis
               )

full_dashboard = pn.Row(pn.Column("# Lets Investigate Politician Stock Trading Together!",
                           "## Select the Area",
                           tabs,
                        #    recruiting,
                        #    linked_dataset
                                  ))
full_dashboard.servable()

Event(what='value', name='value', obj=RadioButtonGroup(options=['Trades Profitability Dis...], value='Average Profit b...), cls=RadioButtonGroup(options=['Trades Profitability Dis...], value='Average Profit b...), old='Trades Profitability Distribution', new='Average Profit by Politician', type='changed') 
Event(what='value', name='value', obj=RadioButtonGroup(options=['Trades Profitability Dis...], value='Trades Profitability D...), cls=RadioButtonGroup(options=['Trades Profitability Dis...], value='Trades Profitability D...), old='Average Profit by Politician', new='Trades Profitability Distribution', type='changed') 
Event(what='value', name='value', obj=RadioButtonGroup(options=['Trades Profitability Dis...], value='Average Profit b...), cls=RadioButtonGroup(options=['Trades Profitability Dis...], value='Average Profit b...), old='Trades Profitability Distribution', new='Average Profit by Politician', type='changed') 
Event(what='value', name='value', obj=RadioButtonGroup(options=['Trades Profitability Dis...], value='Profitability S...), cls=RadioButtonGroup(options=['Trades Profitability Dis...], value='Profitability S...), old='Average Profit by Politician', new='Profitability Scatterplot', type='changed') 
Event(what='value', name='clicks', obj=Button(clicks=1, name='Update Range (x-axis)'), cls=Button(clicks=1, name='Update Range (x-axis)'), old=0, new=1, type='set')

Row
    [0] Column
        [0] Markdown(str)
        [1] Markdown(str)
        [2] Tabs
            [0] Column(name='Transaction C...)
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] Markdown(str)
                    [1] Select(options=['Politician Name', ...], value='Politician Name')
                [2] Row
                    [0] Markdown(str)
                    [1] EditableIntSlider(end=50, start=10, value=10, width=300)
                [3] Row
                    [0] Markdown(str)
                    [1] RadioButtonGroup(options=['Bar Plot', 'Pie Chart'], value='Bar Plot')
                [4] Plotly(Figure, height=600, sizing_mode='stretch_width')
            [1] Column(name='Trade Cycle Analysis')
                [0] Row
                    [0] Markdown(str)
                [1] Row
                    [0] Markdown(str)
                    [1] RadioButtonGroup(options=['Trades Profitability Dis...], value='Trades Profitability D...)
                [2] Row
                    [0] IntInput(end=3200, name='Minimum Value (x)', start=-2900, value=-2900)
                    [1] RangeSlider(end=3200, start=-2900, step=1, value=(-2900, 3200), value_end=3200, value_start=-2900)
                    [2] IntInput(end=3200, name='Maximum Value (x)', start=-2900, value=3200)
                    [3] Column
                        [0] Spacer(height=17)
                        [1] Button(name='Update Range (x-axis)')
                [3] Plotly(Figure, height=600, sizing_mode='stretch_width')